# Modélisation du système bactérie/anticorps

Nadir Ait Kheddache, Idriss Alaoui Souliman, Fabien Benthami, Alma Draeger, Alexandre Janin

In [20]:
#Imports
import numpy as np
import ipywidgets as widgets
import IPython

In [103]:
#Variables globales
#probabilité d'avoir chaque type de cellule (0: vide, 1: bactérie, 2: globule blanc, 3: nutriments)
#la somme des probabilités doit être égale à 1
proba = [0.5, 0.1, 0.1, 0.3]

#probabilité qu'une cellule vide se transforme en nutriments
proba_apparition_nutriments = 0.1

#probabilité qu'un globule blanc dispairaisse à chaque tour
proba_apoptose = 0.01

#nombres de globules blancs qui apparaissent à chaque tour
nb_gb_apparition = 2

#nombres de GB nécessaires pour tuer une bactérie
nb_gb_necessaires = 2

#nombres de GB nécessaires pour qu'un GB apparaisse
nb_gb_reproduction = 4

#nombres de bactéries voisines à partir duquel la bactérie meurt 
nb_b_surpop = 5

#nombres de bactéries voisines à partir duquel le globule blanc meurt 
nb_b_necessaires = 5

#Symbole qui représente chaque valeur de cellule
cara = ['-', 'B', 'G', 'n']

In [104]:
#Paramètres
infiniCheckbox = widgets.Checkbox(value = False, description = "Infini", disabled = False)
display(infiniCheckbox)

Checkbox(value=False, description='Infini')

In [105]:
def matrice(l, h):
    """ int^2 -> int[][]
    largeur > 0, hauteur > 0
    Renvoie une matrice de 0 de l colonnes et h lignes"""
    
    return [[aleatoire() for x in range(l)] for y in range(h)]

In [106]:
def aleatoire():
    """float -> int
    Renvoie le type de cellule (0, 1, 2, 3) selon le nombre aléatoire tiré"""
    
    alea = np.random.rand()
    
    for i in range(len(proba)):
        p = proba[i]
        if alea <= p:
            return i
        else:
            alea -= p
            
    return 0

In [107]:
def printmatrice(M):
    """int[][] -> NoneType
    Affiche la matrice M"""
    IPython.display.clear_output()
    for ligne in M:
        s = ""
        for cell in ligne:
            s += cara[cell] + ' '
        print(s + "\n")

In [108]:
def voisinage(M, x, y):
    """int[][] * int * int * bool-> int[]
    Renvoie la liste des voisins de la case (x, y).
    Si infini est False, on ne prend pas les voisins hors de la matrice"""
    v = [0, 0, 0, 0]
    
    ymax = len(M)
    xmax = len(M[0])
    
    infini = infiniCheckbox.value
    
    for i in [x - 1, x, x + 1]:
        if (0 <= i and i < xmax):
            for j in [y - 1, y, y + 1]:
                if (0 <= j and j < ymax):
                        if (i, j) != (x, y): #Si i, j n'est pas la case centrale
                            v[M[j][i]] += 1
    return v

In [109]:
def next_step_matrice(M):
    """int[][] -> int [][]
    Renvoie la matrice à l'étape suivante."""
    
    nv_M = list(M)
    
    for y in range(len(M)):
        for x in range(len(M[y])):
            nv_M[y][x] = next_step_cellule(M[y][x], voisinage(M, x, y))
            
    return nv_M

In [110]:
def next_step_cellule(c, voisins):
    """int * int[] -> int
    Hypothèse: c = 0 ou 1 ou 2 ou 3
    Renvoie l'état suivant d'une cellule c en fonction de son état actuel et de ses voisins."""
    
    vides = voisins[0]
    bacteries = voisins[1]
    globules = voisins[2]
    nutriments = voisins[3]
    
    
    if (c == 0): #La cellule est vide
        if (globules >= nb_gb_reproduction): #Un nouveau GB est créé
            return 2
        elif (np.random.rand() <= proba_apparition_nutriments): #Un nutriment apparait
            return 3
        else: #La cellule reste vide
            return 0
        
    if (c == 1): #Bactérie
        if (globules >= nb_gb_necessaires): #Les GB tuent la bactérie
            return 0
        elif(bacteries >= nb_b_surpop): #La bactérie meurt (surpopulation)
            return 0
        else: #La bactérie survit
            return 1
        
    if (c == 2): #Globule blanc
        if (bacteries >= nb_b_necessaires): #Il y a trop de bactéries, le GB meurt
            return 0
        elif (np.random.rand() <= proba_apoptose): #Le GB meurt par apoptose
            return 0
        else: #Le GB survit
            return 2
        
    if (c == 3): #Nutriment
        if (bacteries > 0): #La bactérie voisine se multiplie
            return 1
        else: #La cellule reste un nutriment
            return 3
        
    print("Erreur: c invalide:", c)

In [111]:
mat = matrice(16,8)

In [112]:
def display_next_step(b):
    global mat
    global button
    global reset_button
    
    mat = next_step_matrice(mat)
    printmatrice(mat)
    
    display(reset_button)
    display(button)

In [113]:
def reset(b):
    global mat
    mat = matrice(16, 8)
    printmatrice(mat)
    display(reset_button)
    display(button)

In [114]:
reset_button = widgets.Button(
    description='Reset',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me!',
    icon='check'
)

button = widgets.Button(
    description='Next Step',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me!',
    icon='check'
)

button.on_click(display_next_step)
reset_button.on_click(reset)

display(reset_button)
display(button)

B B B B B B B B B B B B B B B B 

- - - - - - - - - B - - - - B B 

B - B B B B B B B - B B B B - B 

B - B - - - B n B B - B - B - B 

B - - - B B - - - n - B n B B B 

B B B B - B B B B B - B - B - B 

- - - - - - - - B - n B - - - B 

B B B B B B B B B B B B B B B B 



Button(button_style='info', description='Reset', icon='check', style=ButtonStyle(), tooltip='Click me!')

Button(button_style='info', description='Next Step', icon='check', style=ButtonStyle(), tooltip='Click me!')

In [69]:
def simulation(l, h, steps):
    """int -> NoneType
    Simule steps étapes et affiche le résultat final"""